# [Puntajes PSU](http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016)

In [ ]:
from robobrowser import RoboBrowser
import geocoder
import folium
import pandas
import os
from utils import persist_to_file
cwd = os.getcwd()

browser = RoboBrowser(history=True, parser='html5lib')
url = 'http://www.t13.cl/noticia/nacional/te-puede-servir/ranking-colegios-mejor-promedio-psu-2016'
browser.open(url)

In [ ]:
table=browser.select('.article-content table')[0]
t=table.__str__()
t[:1000]

In [ ]:
t=t.replace(',','.')
df=pandas.read_html(t, header=0)[0]
df.head(10)

In [ ]:
g=geocoder.google('colegio the grange school, la reina, chile')
print(g)
print(g.latlng)

In [ ]:
g.latlng

[-36.8883627, -73.0351594](
https://www.google.cl/maps/@-33.4382165,-70.5686165,3a,75y,3.98h,101.74t/data=!3m7!1e1!3m5!1sxP13kKF1tvm6pau-DNnsPA!2e0!6s%2F%2Fgeo1.ggpht.com%2Fcbk%3Fpanoid%3DxP13kKF1tvm6pau-DNnsPA%26output%3Dthumbnail%26cb_client%3Dmaps_sv.tactile.gps%26thumb%3D2%26w%3D203%26h%3D100%26yaw%3D33.012394%26pitch%3D0%26thumbfov%3D100!7i13312!8i6656)

In [ ]:
import time

@persist_to_file()
def get_latlng(query):
    providers = geocoder.osm, geocoder.google
    for provider in providers:
        #print(provider)
        try:  
            g=provider(query)
        except ValueError:
            continue
        if g:
            return g.latlng
    return None, None

for index,row in df.iterrows():
    stopwords = 'colegio', 'liceo', 'school', 'instituto', 'college'
    query = row.Colegio +", "+ row.Comuna + ", chile"
    query = query.lower()
    for i in stopwords:
        if i in query:
            break
    else:
         query = 'colegio ' + query
    print(query),
    g=get_latlng(query)
    time.sleep(0.1)
    if not g:
        print('ERROR', query)
        continue
    if g:
        lat, lng = g
        df.set_value(index,'lat', lat)
        df.set_value(index,'lng', lng)
        print(g)
    else:
        print(g, 'ERROR', query)

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df=df.dropna()
df.head(25)

In [ ]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Terrain')
i=0
for index, row in df.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#000000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5, 
                            popup=u"%s %s %s" % (row['Colegio'], row['Promedio PSU'], row['Comuna'])))
map.save('colegios.html')
map

In [ ]:
df.head()

In [ ]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='Stamen Toner')
i=0
for index, row in df.iterrows():
    map.add_child(
        folium.CircleMarker([row.lat,row.lng],
                            color='#ff0000',
                            fill_color='#ff0000',
                            fill=True,
                            fill_opacity=0.7,
                            radius=5*row['Alumnos que rindieron la prueba']/100.0, 
                            popup=u"%s %s %s %s alumnos" % (row['Colegio'], row['Promedio PSU'], row['Comuna'], row['Alumnos que rindieron la prueba'])))
map.save('colegios.html')
map

In [ ]:
df.tail(20)

In [ ]:
df3 = df[df['Tipo de establecimiento'] != 'Particular']
df3

In [ ]:
map = folium.Map(location=[-33.39, -70.57], zoom_start=8, tiles='OpenStreetMap')
colors = {
    'Particular': ('green', ''),
    'Subvencionado': ('red', 'star'),
    'Municipal': ('red', 'star-empty'),
}
for index, row in df.iterrows():
    color, icon = colors[row['Tipo de establecimiento']]
    folium.Marker(
        location=[row.lat,row.lng],
        popup=u"%s - %s - %s - %s" % (row['Tipo de establecimiento'], row.Colegio, row['Promedio PSU'], row['Comuna']),
        icon=folium.Icon(color=color, icon=icon)
    ).add_to(map)
map.save('colegios.html')
map

[mapa](colegios.html)